In [1]:
# モデルを読み込む
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)


c:\Users\shinta\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import csv

# CSVファイルを読み込む
# https://w1740803485-clv347541.slack.com/archives/C08F7JZPD63/p1745221416748989?thread_ts=1745208882.923669&cid=C08F7JZPD63
# https://github.com/digitaldemocracy2030/kouchou-ai/issues/280
source_table = []
with open('kankyou_pubcom.csv', 'r', encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        source_table.append(row)

header = source_table[0]
source_table = source_table[1:]

# データ構造は要約、本文の順
source_table[:10]


[['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進してください。'],
 ['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください。行政は風評と感情論に屈しないで妥協しないで欲しい。何を説明しても感情論では解決できない。科学で解決して欲しい。'],
 ['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるために必要な改正と理解します。再生利用の必要性が国民に理解され、事業が円滑に進むことを期待します。'],
 ['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去土壌の放射能濃度、復興再生利用に係る工事の施工及び維持管理に関する基本的な事項等が明確になり、福島の復興が促進されることとなると考えられる。除去土壌の処分については、全国的な取り組みが不可欠であり、国が積極的に主導して進めていただきたい。'],
 ['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業の安全性やコストなどの点からは福島県内で完結させた方が好ましいのかもしれませんが、（沖縄の基地問題同様に）特定の地域に負担を押し付ける構造となり、国内の分断を招くと思います。福島の原発事故以前に有権者であった私を含め、多くの国民は原発を容認してしまった過去があり、原発事故については一定の責任があると考えています。除去土壌を、福島以外の様々な地域にて処理することは、国全体の責任として受け入れる必要があると思っています。'],
 ['１ 福島の復興のため、復興再生利用等を推進すべき。',
  '・ 中間貯蔵施設の除去土壌について「福島県外での最終処分や再生利用」などについて賛成です。福島に作られた中間貯蔵施設は決して「最終処分場」にはならないということが法律にも明記されています。「除染土壌」を「放射能汚染土」

In [11]:
# source_table は id, カテゴリ番号、要約, 本文の構造
# idと本文 だけを抽出する、重複を排除する

already_seen = set()
already_seen.add("") # 空白だけのコメントを除去するために、予め追加しておく

target_table = []
for i, row in enumerate(source_table):
    id = i
    data = " ".join(row[1:]) # カンマで繋がっている可能性があるので、joinしておく
    if data not in already_seen:
        already_seen.add(data)
        target_table.append((id, data))
target_table[:3]

[(0, '・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進してください。'),
 (1,
  '・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください。行政は風評と感情論に屈しないで妥協しないで欲しい。何を説明しても感情論では解決できない。科学で解決して欲しい。'),
 (2,
  '・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるために必要な改正と理解します。再生利用の必要性が国民に理解され、事業が円滑に進むことを期待します。')]

In [ ]:
# エンベデッドする
sentences = [row[1] for row in target_table]
embeddings = model.encode(sentences)
print(embeddings.shape)

In [7]:
# 埋め込みベクトルから、コサイン類似度行列を算出する
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.4395, 0.7499,  ..., 0.4361, 0.4711, 0.5236],
        [0.4395, 1.0000, 0.4821,  ..., 0.4033, 0.3204, 0.4700],
        [0.7499, 0.4821, 1.0000,  ..., 0.3203, 0.4573, 0.5933],
        ...,
        [0.4361, 0.4033, 0.3203,  ..., 1.0000, 0.5693, 0.2718],
        [0.4711, 0.3204, 0.4573,  ..., 0.5693, 1.0000, 0.2585],
        [0.5236, 0.4700, 0.5933,  ..., 0.2718, 0.2585, 1.0000]])


In [ ]:
# 類似度が高いものを抽出
import numpy as np

# score, text1, text2 の順で格納
similar_texts = []

# 類似度が0.8以上のインデックスを取得
high_similarity_indices = np.where(0.8 < similarities)

# 高い類似度のペアを抽出
for idx in range(len(high_similarity_indices[0])):    
    i, j = high_similarity_indices[0][idx], high_similarity_indices[1][idx]
    if j >= i:
        continue

    score = float(similarities[i, j])
    similar_texts.append((score, target_table[i][0], sentences[i], target_table[j][0], sentences[j]))

# 類似度でソート
similar_texts.sort(key=lambda x: x[0], reverse=True)
similar_texts[:30]

[(0.9937520027160645,
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対反対絶対反対絶対反対絶対反対絶対',
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対反対絶対反対絶対'),
 (0.9845615029335022, '２ 反対。（理由の記載なし）', '・ 反対反対', '２ 反対。（理由の記載なし）', '・ 反対'),
 (0.9570755958557129,
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対反対絶対反対絶対',
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対的'),
 (0.9415979385375977,
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対反対絶対反対絶対反対絶対反対絶対',
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対的'),
 (0.9242745041847229, '２ 反対。（理由の記載なし）', '・ 断固反対です。', '２ 反対。（理由の記載なし）', '・ 反対'),
 (0.9222958087921143,
  '２ 反対。（理由の記載なし）',
  '・ 反対反対',
  '２ 反対。（理由の記載なし）',
  '・ 断固反対です。'),
 (0.9202225208282471,
  '２ 反対。（理由の記載なし）',
  '・ 反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対反対',
  '２ 反対。（理由の記載なし）',
  '・ 反対反対反対反対反対反対反対'),
 (0.9201971292495728,
  '13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。',
  '・ 放射能汚染土を全国にばら撒くな。',
  '２ 反対。（理由の記載なし）',
  '・ 放射能汚染土の再利用はやめてください！'),
 (0.910454511642456,
  '２ 反対。（理由の記載なし）',
  '・ 反対絶対反対絶対的',
  '２ 反対。（理由の記載なし）',
  '・ 断固反対です。'),
 (0.9097589254379272,
  '2

In [10]:
# similar_textsをCSVに書き出す
import pandas as pd
similar_texts_df = pd.DataFrame(similar_texts, columns=['score', 'id1', 'text1', 'id2', 'text2'])
similar_texts_df.to_csv('similar_texts_kankyou.csv', index=False, encoding='utf-8-sig')